# TripAdvising Through SEAsia (under construction)

### Summary

<ul>

<li>Tourist attractions are governed by power law-like distributions</li>
<li>Reviews can be mined to find idiosyncratic characteristics of tourist attractions.</li>
<li> The sparseness (&lt;0.01%), lack of rating variance (avg rating 4.4/5), and low user participation (2 reviews/user) of this partial TripAdvisor dataset does not lend itself well to a recommender system. Systems with user stickiness (Netflix, Amazon) where user feedback can be rapidly captured would likely be more amenable</li>
<li>More useful might be a search feature</li>



</ul>

### Introduction

After graduating, I took some time off to travel the world.  One area that I particularly enjoyed visiting was Southeast Asia.  So for a side project, I decided to scrape the TripAdvisor website for all the attractions reviews in (Cambodia, Laos, Vietnam) and then mine the data to for insights.

<b>Data</b>
<ul>
<li>430k reviews, 200k users, 5k reviewed items at 800MB.</li>
</ul>

<b>Processing</b>
<ul>
<li>Scraped, cleaned, and processed with Python</li>
<li>Stored in SQLite</li>
<li>Visualizations done in R</li>
</ul>

### Non-normal distributions govern TripAdvisor reviews

Immediately, we notice that the dataset is extremely sparse (0.03%).  In comparison, in 2006, when Netflix held a competition to improve their recommendation algorithm, the sparsity of that dataset was 1%, a 30x difference.

<img src="figs/reviews_item_histo.png" style="max-height: 400; max-width: 600px;">

By plotting the distribution of reviews for attractions, we find that it is governed by a power law-like distribution with a very long tail.  A large number of attractions have a handful of reviews while few attractions have a large number of reviews.  

One plausible explanation is that when dealing with popularity, there is a positive feedback loop, popular items become more popular.  If you have a week of vacation, you are likely to spend it visiting the most popular attraction than the 50th most popular.  This phenomenon of preferential attachment is widely known under many names such as Zipf's law, Pareto principle, and the Matthew effect.

<img src="figs/reviews_user_histo.png" style="max-height: 400; max-width: 600px;">

We also find that in this dataset, users overwhelmingly only rate a single attraction.  There is noticeable a lack of stickiness; users don't keep rating attractions, it seems to be a one-off endeavour.

<img src="figs/city_popularity.png" style="max-height: 400; max-width: 600px;">

A similar distribution is observed in the number of reviews per city.  The highest number of reviews belong to the town of Siem Reap, home to the world famous Angkor Wat. There is also a high number of reviews for the most populated urban areas (Ho Chi Minh City, Hanoi) and a popular tourist town (Hoi An) with a subsequent steep drop-off and long tail of smaller many tourist attractions.

### English as lingua franca

Next, I was wondered how well my observations while traveling were reflected in the data.

The first general observation was that English was the de facto tourist lingua franca everywhere.  The tourist infrastructure (signs, menus, hotels, etc.) in all the countries I visited were in English. As a result, Chinese tourists negotiating bike rentals with Thais in broken English was a common occurence.  With the large caveat that TripAdvisor may be more popular in English speaking countries, the data reflects the dominance of English by showing the large share of reviews in English. 

The cities of Nha Trang and Phan Thiet (Mui Ne) cities stand out in the proportion of Russian reviews as a consequence of being well-known as tropical get-aways for Russian tourists.

There are a fair bit of French reviews as well, this is unsurprising as large parts of Southeast Asia was formerly known French Indochina.

<img src="figs/location_lang_histo.png" style="max-height: 400; max-width: 600px;">

### Where should I be careful?

In Vietnam, thieves would typically either slash a woman's purse in the marketplace or do a drive-by motorcycle snatching.  The other favored approach was to wait until tourists were in the ocean and then steal their valuables from the beach or the hotel.

Most tourist websites may mention theft, but it is rarely quantified.  However, with the TripAdvisor dataset, it may be possible to estimate the comparative frequency of theft.

Reviews were filtered for stop words and stemmed.  Reviews with instances of words that had a strong relation to theft (pickpocket, stole) were marked as a theft-related review and counted.

For a benchmark, the American city of Baltimore was included.

<img src="figs/pickpocket_cities_histo.png" style="max-height: 400; max-width: 600px;">


<b>Lock up your valuables while at the beach and in busy cities!</b>

The top three cities with reviews mentioning theft are popular touristy beach towns while the next four cities are the major cities of the region.  Notably, the mention of theft in all these locations are much greater when compared to Baltimore.  This aligns well with my personal experiences.  At one point while in Ho Chi Minh City, roughly 20% of my hostel had been victims of some sort of theft!

Are there specific places to be wary of?

<img src="figs/pickpocket_locs_histo.png" style="max-height: 400; max-width: 600px;">

<b>Thieves like busy markets!</b>

With the notable exception of the Why Not Bar, theft is most often mentioned in the reviews of markets in busy cities.  Notably, the locations listed for Ho Chi Minh City are the major backpacker areas.  All three areas are within ~500m of each other and favorites of thieves on motorcycles and pickpockets. The inclusion of the Angkor National Museum is likely due to users commenting on cultural artefacts that were stolen.

How about scams? The bane of any tourist's existence.

<img src="figs/scam_locs_histo.png" style="max-height: 400; max-width: 600px;">

<b>Angkor What?</b>

The top 5 out of 6 are reviews of the Tonle Sap lake nearby Angkor Wat.  It's interesting that it is so localized.

### A recommendation engine

Having the frustration of having Tripadvisor recommend a lot of temples and inspired by the Netflix prize, I wanted to see if I could build a better recommendation engine.

<b>Everything is awesome!</b>

A histogram of the tripadvisor ratings show that it is skewed towards the upper end.  Tourists are very satisfied.  The average rating is a high 4.4/5 stars.  This is in contrast to the <a href="http://www.timelydevelopment.com/demos/NetflixPrize.aspx">Netflix data</a>, where the average rating was 3.5/5 stars and roughly normally distributed.  

<img src="figs/ratings_histo.png" style="max-height: 400; max-width: 600px;">

Intuitively, this makes sense.  Movies incur a one time upfront cost for production.  Distribution costs of digital goods are neglible and shelf space isn't an issue so Netflix can house an almost infinite selection. Therefore, a bad movie on Netflix can live in perpetuity.

In contrast, tourist attractions are dynamic. If tourists dislike an attraction, the operators will eventually go out of business; thus, there is a positive selection bias in the attractions listed on TripAdvisor.  One would probably observe a similar effect in industries where the customer is looking for a one-time great experience - amusement parks, safaris, etc. How common are mediocre amusement parks?

### General Framework

Despite the perceived difficulties, I was curious to see if algorithmically beating a dumb baseline was possible.  Similar to the Netflix prize competition, the objective function was to minimize the RMSE. A cross-validation split of 80/20 was used with repeated random sub-sampling to determine model RMSE values.

### Models

The following algorithms are detailed in a  <a href="http://public.research.att.com/~volinsky/netflix/kdd08koren.pdf">paper</a> written by the winners of the Neflix competition.  All models were optimized using stochastic gradient descent with L2 regularization.

#### Simple Predictors
A rating $r_{u,i}$ given by user $u$ for item $i$ can be modeled with simple biases for both the user $b_{u}$ and the item $b_{i}$:
$$ r_{u,i} = \mu + b_{u} + b_{i}, $$

where $\mu$ is the overall average rating.
#### Singular Value Decomposition (SVD)

With SVD, the goal is to uncover latent features that explain the dataset.  Each user $u$ is associated with a vector $p_{u}$ and each item with an items-factor vector $q_{i}$.  The inner product is then the estimate:

$$r_{u,i} = q_{i}^{T}p_{u} $$

#### Improved SVD
$$r_{u,i} = q_{i}^{T}p_{u} +  \mu + b_{u} + b_{i} $$

Improved SVD is a blend of of the simple predictors and the plain SVD.

### Results

<img src="figs/model_performance.png" style="max-height: 400; max-width: 600px;">

We find that SVD model fares the best with a 3% improvement over recommending the average.  Interestingly, the blend of baseline predictors and SVD did not fare as well as either of the individual predictors due to possible overfitting.

However, it's debatable whether further model optimization is worthwhile.  Given that the 97% of users rate < 5 items and the previously mentioned difficulties of the dataset, there is the inevitably of diminishing returns to effort.

### Conclusions

After reviewing the collected TripAdvisor dataset, there seems to be limited gains from implementing a recommender system based purely on ratings. 

However, the dataset is very rich in reviews and can be mined for insights into idiosyncratic characteristics of tourist attractions.  To that end, a better product involving this dataset may be a search function designed to look for locations that aren't categorized into preset TripAdvisor categories.